In [3]:
import pandas as pd
import numpy as np

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data=pd.read_csv('/content/drive/MyDrive/iot-dataset/device_4_dataset.csv')

In [5]:
data.head()
data.groupby('type')['type'].count()

type
benign            175240
gafgyt_combo       58152
gafgyt_junk        28349
gafgyt_scan        27859
gafgyt_tcp         92581
gafgyt_udp        105782
mirai_ack          91123
mirai_scan        103621
mirai_syn         118128
mirai_udp         217034
mirai_udpplain     80808
Name: type, dtype: int64

In [6]:
data["type"].replace({"gafgyt_combo": "attack" ,
                             "gafgyt_junk" : "attack",
                             "gafgyt_scan" : "attack",
                             "gafgyt_tcp" : "attack",
                             "gafgyt_udp" : "attack",
                             "mirai_ack" : "attack",
                             "mirai_scan" : "attack",
                             "mirai_syn" : "attack",
                             "mirai_udp" : "attack",
                             "mirai_udpplain" : "attack"},
                             inplace=True)

In [7]:
x = data.iloc[:,:-1].values
y = data.iloc[:,-1].values

In [ ]:
#standardize numerical columns
# def standardize(df,col):
#     df[col]= (df[col]-df[col].mean())/df[col].std()
# data_st=data.copy()
# for i in (data_st.iloc[:,:-1].columns):
#     standardize (data_st,i)
# data_st.head()

In [8]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)

In [9]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x = sc.fit_transform(x)

In [10]:
x = x.reshape(x.shape[0], x.shape[1], 1)
print(x.shape)

(1098677, 115, 1)


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2,shuffle=True)

In [12]:
from keras import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D

model = Sequential()
model.add(Conv1D(64, 2, activation="relu", batch_input_shape=(None, 115, 1)))
model.add(Dense(16, activation="relu"))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(2, activation = 'softmax'))
model.compile(loss = 'sparse_categorical_crossentropy', 
     optimizer = "adam",               
              metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 114, 64)           192       
                                                                 
 dense (Dense)               (None, 114, 16)           1040      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 57, 16)           0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 912)               0         
                                                                 
 dense_1 (Dense)             (None, 2)                 1826      
                                                                 
Total params: 3,058
Trainable params: 3,058
Non-trainable params: 0
______________________________________________________

In [13]:
#Fitting the data to the training dataset
model.fit(X_train,y_train, validation_data=(X_test,y_test), epochs=1)

27467/27467 [==============================] - 227s 8ms/step - loss: 0.0027 - accuracy: 0.9994 - val_loss: 0.0012 - val_accuracy: 0.9997


In [14]:
y_pred=model.predict(X_test)
y_pred = y_pred.argmax(axis=-1)

In [15]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, matthews_corrcoef, f1_score, precision_score, recall_score

print(f"Accuracy   : {accuracy_score(y_test, y_pred)}")
print(f"Precision  : {precision_score(y_test, y_pred)}")
print(f"F1-Measure : {f1_score(y_test, y_pred)}")
print(f"MCC        : {matthews_corrcoef(y_test, y_pred)}")
print(f"Recall     : {recall_score(y_test, y_pred)}")

Accuracy   : 0.9997496996395675
Precision  : 0.9997412454717958
F1-Measure : 0.9992097814686571
MCC        : 0.9990612803207464
Recall     : 0.9986788822194779


In [16]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    184917
           1       1.00      1.00      1.00     34819

    accuracy                           1.00    219736
   macro avg       1.00      1.00      1.00    219736
weighted avg       1.00      1.00      1.00    219736



In [17]:
print(confusion_matrix(y_test, y_pred))

[[184908      9]
 [    46  34773]]
